<a href="https://colab.research.google.com/github/waltz2u/bd/blob/master/DeepLearning3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounting Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Needs to crop so need to get the min size

In [2]:
from PIL import Image
import glob
import pickle
import functools as ft
import numpy as np
image_sizes = []
for filename in glob.glob('/content/gdrive/My Drive/models/3d/*.jpg'): #assuming jpg
    im = Image.open(filename)
    image_sizes.append(im.size)    

min_size = list(ft.reduce(lambda x,y: [min(x[0],y[0]), min(x[1],y[1])] , image_sizes))  
print(min_size)  

[701, 341]


Crop all the images then store 

In [0]:
area = (0,0,min_size[0],min_size[1])  
images_list = np.empty(shape=[len(image_sizes),min_size[0], min_size[1], 3])
i = 0
for filename in glob.glob('/content/gdrive/My Drive/models/3d/*.jpg'): #assuming jpg
    im = Image.open(filename)
    im = im.crop(area)
    #tt = np.array(im)
    #print(tt.shape)
    #break
    image = np.transpose(np.array(im),[1,0,2])
    #print(image.shape)
    #break
    images_list[i] = image
    i += 1
    #images_list.append(im.getdata())   
    #in_data = np.asarray(images_list[0].getdata())
    #in_data.T


Splitting training/test

In [0]:
import random
num_classes = 3
sample_size = len(images_list)
sample_size_class = round(len(images_list)/num_classes)
training_split = 0.7
k = round(training_split * sample_size_class)
m = sample_size_class - k
training_index = random.sample(range(sample_size_class), k) + random.sample(range(sample_size_class, 2*sample_size_class), k) + random.sample(range(2*sample_size_class, 3*sample_size_class), k) 
test_index = [e for e in range(3*sample_size_class) if e not in training_index]
# images_list_array = np.asarray(images_list)

In [15]:
len(training_index)

189

In [0]:
x_train, y_train = images_list[training_index], [0]*k + [1]*k + [2]*k
x_test, y_test = images_list[test_index], [0]*m + [1]*m + [2]*m

In [17]:
x_train.shape

(189, 701, 341, 3)

In [0]:
import keras

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

The model

In [0]:
import warnings
warnings.filterwarnings('ignore')

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(min_size[0], min_size[1],3) ))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

batch_size = 128
num_classes = 10
epochs = 12

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

pickle.dump(model, open('cnn/dl_estimator.sav', 'wb'))

In [0]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])